In [40]:
import geopy
from geopy.geocoders import Nominatim
import requests
import folium
import itertools
import pandas as pd
import networkx as nx

## INITIALIZE DATA

In [13]:
cast = {'drivers': {'Laci': {'address': '1085 Budapest Horánszky utca 6.', 'capacity': 2},
                    'Feri': {'address': '1153 Budapest Dobó utca 18.', 'capacity': 2}
                   },
        'cast_members': {'Bius': {'address': '1077 Budapest Wesselényi utca 58.'},
                         'Zoli': {'address': '1072 Budapest Akácfa utca 4.'},
                         'Peti': {'address': '1155 Budapest Perczel Mór utca 13.'},
                         'Nori': {'address': '1142 Budapest Szőnyi út 2.'}
                        },
        'destination': {'Shooting': {'address': '1021 Budapest Budakeszi út 51'}}
       }

In [4]:
geolocator = Nominatim(user_agent="Transport_planner")

In [5]:
def geocode_address(address, geolocator=geolocator):
    return geolocator.geocode(address)

In [6]:
laciloc = geolocator.geocode(drivers['Laci']['address'])
print(laciloc.latitude, laciloc.longitude)

47.49236565 19.0677840526337


In [7]:
feriloc = geolocator.geocode(drivers['Feri']['address'])
print(feriloc.latitude, feriloc.longitude)

47.5604898333333 19.1154243333333


## DIRECTIONS API

In [8]:
with open('gitignore/api_key.txt') as fh:
    api_key = fh.read()

In [9]:
url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={api_key}&start={str(laciloc.longitude)[:8]},{str(laciloc.latitude)[:8]}&end={str(feriloc.longitude)[:8]},{str(feriloc.latitude)[:8]}"
url

'https://api.openrouteservice.org/v2/directions/driving-car?api_key=5b3ce3597851110001cf6248934a0ac43b7847269eec224b84b6f488&start=19.06778,47.49236&end=19.11542,47.56048'

In [21]:
def get_duration(loc_1, loc_2, api_key=api_key):
    # return time of route plan in seconds
    geojson = get_directions(loc_1, loc_2, api_key=api_key)
    return geojson['features'][0]['properties']['segments'][0]['duration']

def get_directions(loc_1, loc_2, api_key=api_key):
    # get direction as a geojson
    url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={api_key}&start={str(loc_1.longitude)},{str(loc_1.latitude)}&end={str(loc_2.longitude)},{str(loc_2.latitude)}"
    r = requests.get(url)
    return r.json()

In [22]:
locations = {}
for k, v in cast.items():
    for key, value in v.items():
        locations[f'{k}_{key}'] = geocode_address(value['address'])

In [35]:
relations = [element for element in itertools.product(*[locations.keys(), locations.keys()]) if (
    (element[0] != element[1]) &
    (not element[0].startswith('drivers') & element[1].startswith('drivers')))]


In [122]:
edgelist[edgelist['start'] == 'drivers_Laci']

,start,end,dist
0,drivers_Laci,cast_members_Bius,237.9
1,drivers_Laci,cast_members_Zoli,132.0
2,drivers_Laci,cast_members_Peti,840.4
3,drivers_Laci,cast_members_Nori,837.9
4,drivers_Laci,destination_Shooting,1197.4


## CREATE GRAPH

In [37]:
graph_edges = []
for first, second in relations:
    if not (second, first) in graph_edges:
        graph_edges.append((first, second))

In [41]:
edgelist = pd.DataFrame(graph_edges, columns = ['start', 'end'])

In [46]:
def get_distance(row):
    return get_duration(locations[row['start']], locations[row['end']])

In [48]:
edgelist['dist'] = edgelist.apply(get_distance, axis = 1)

In [54]:
vertices = list(set(list(edgelist['start'].unique()) + list(edgelist['end'].unique())))

In [62]:
vertices = {i: {'name': vertices[i]} for i in range(len(vertices))}

In [70]:
vertex_indices = {v['name']: k for k, v in vertices.items()}

In [100]:
edges = [(vertex_indices[row['start']], vertex_indices[row['end']], row['dist']) for i, row in edgelist.iterrows()]

In [113]:
G = nx.Graph()

In [115]:
for k, v in vertex_indices.items():
    G.add_nodes_from([v], name=k)

In [106]:
G.add_weighted_edges_from(edges)

In [ ]:
m = folium.Map(location=(feriloc.latitude, feriloc.longitude))

folium.GeoJson(
    geojson,
    name='geojson'
).add_to(m)

In [ ]:
m